Alcohol Consumption and Average Grades
--------------------------------------


----------


This notebook examines whether alcohol consumption has any predictive power over student average grades. I am also interested in learning what other features may be important predictors of student grades. Just a word of warning, I am not going to attempt to predict student grade evolution over marking periods since all the features in the dataset (other than grades) remain constant over marking periods and are general descriptors of student backgrounds. My results indicate that while alcohol consumption on weekdays and weekends are not the strongest predictor of student average grades, they are in the top 10 (out of 29). The two strongest predictors of student average grade are the willingness to pursue higher education (higher) and (guess what?) mother's education (Medu).


----------


The notebook is organized as follows. I first conduct some EDA to see if the two tables (with math and Portuguese grades) can be combined. After combining the two tables, I obtain student average grades over marking periods and estimate 3 models: linear, regression tree, and random forest. Some quick cross-validation indicates that the random forest is the best performing model out of the three. Finally, I use the best-performing random forest to rank features by their importance.

First, I would ideally like to combine both tables into one. But before that I would like to get a feeling if math and Portuguese grades are comparable. There is a number of students who are repeated in both tables, so I would use their records to examine if math and Portuguese grades are comparable.

In [ ]:
library(ggplot2)
library(plyr)
library(dplyr)
library(gridExtra)
library(alluvial)
library(extrafont)

d1=read.table("../input/student-mat.csv",sep=",",header=TRUE)
d2=read.table("../input/student-por.csv",sep=",",header=TRUE)

#Following the suggestion of Carlo Ventrella, one of the attributes, "paid," is course specific 
#rather than student specific, so I am eliminating it from the list of attributes by which student
# are matched matched
data.source=merge(d1,d2,by=c("school","sex","age","address","famsize","Pstatus",
                            "Medu","Fedu","Mjob","Fjob","reason","nursery","internet",
                            "guardian","guardian","traveltime","studytime","failures",
                            "schoolsup","famsup","activities","higher","romantic",
                            "famrel","freetime","goout","Dalc","Walc","health","absences"))
print(nrow(data.source)) # 85 students

There are 85 students who belong to both tables, and I am going to examine their average math and Portuguese grades a test case.

In [ ]:
data.source$mathgrades=rowMeans(cbind(data.source$G1.x,data.source$G2.x,data.source$G3.x))
data.source$portgrades=rowMeans(cbind(data.source$G1.y,data.source$G2.y,data.source$G3.y))

data.source$Dalc <- as.factor(data.source$Dalc)      
data.source$Dalc <- mapvalues(data.source$Dalc, 
                              from = 1:5, 
                              to = c("Very Low", "Low", "Medium", "High", "Very High"))

str1=ggplot(data.source, aes(x=mathgrades, y=portgrades)) +
 geom_point(aes(colour=factor(Dalc)))+ scale_colour_hue(l=25,c=150)+
geom_smooth(method = "lm", se = FALSE)

data.source$Walc <- as.factor(data.source$Walc)      
data.source$Walc <- mapvalues(data.source$Walc, 
                              from = 1:5, 
                              to = c("Very Low", "Low", "Medium", "High", "Very High"))

str2=ggplot(data.source, aes(x=mathgrades, y=portgrades))+
geom_point(aes(colour=factor(Walc)))+ scale_colour_hue(l=25,c=150)+
geom_smooth(method = "lm", se = FALSE)

grid.arrange(str1,str2,nrow=2)

The two scatter plots have few implications. First, among the 85 students, no one consumed high or very high levels of alcohol on daily basis. Second, almost all of those who earned relatively high scores consumed very low levels of alcohol on weekdays. Third, math and Portuguese grades seem to correlate highly with each other. When I regress Portuguese grades on math grades, the adjusted R-squared is 0.55. This means that the correlation coefficient between math and Portuguese grades is about 0.74 and that about 55% of the variation in Portuguese grades can be explained by the variation in math grades. In my view, this is an indication that I can go ahead and combine the two tables together without worrying much about the subject matter, average grades in math or Portuguese reflect general student aptitude.  That's what I will do next

In [ ]:
d3<-rbind(d1,d2) #combine the two datasets
# and eliminate the repeats:
d3norepeats<-d3 %>% distinct(school,sex,age,address,famsize,Pstatus,
                Medu,Fedu,Mjob,Fjob,reason,
                guardian,traveltime,studytime,failures,
                schoolsup, famsup,activities,nursery,higher,internet,
                romantic,famrel,freetime,goout,Dalc,Walc,health,absences, .keep_all = TRUE)
#add a column with average grades (math or Portuguese, whichever is available)
d3norepeats$avggrades=rowMeans(cbind(d3norepeats$G1,d3norepeats$G2,d3norepeats$G3))
# and drop grades in 3 marking periods.
d3norepeats<-d3norepeats[,-(31:33)]

Now a basic boxplot of average subject grades grouped by the levels of daily alcohol consumption. 

In [ ]:
ggplot(d3norepeats, aes(x=Dalc, y=avggrades, group=Dalc))+
  geom_boxplot()+
  theme(legend.position="none")+
  scale_fill_manual(values=waffle.col)+
  xlab("Daily Alcohol consumption")+
  ylab("Average Grades")+
  ggtitle("Average Grade")

The median average grade is visually higher among those students who had very low levels of daily alcohol consumption.  However, the median grade of the students with medium, high, and very high levels of daily  alcohol consumption doesn't seem to be very different.  As my first stab at the predictability of average grades using all other variables, I am going to 1) run a multiple linear regression and 2) build a regression tree of average grades on all other variables.

Variable "failures: is closely related to my target variable, avggrades. Since past failures and avggrades represent the same general student aptitude (thus it is rather a target rather than a feature), I am inclined to remove variable "failures" from the dataset.

In [ ]:
failureind<-which(names(d3norepeats)=="failures")
d3norepeats<-d3norepeats[,-failureind]

In [ ]:
# 1) multiple regression 
lm2<-lm(avggrades~., data=d3norepeats[,1:30])
summary(lm2)

Adjusted R-squared in the above regression is only 0.17, which is quite low. It implies that only 17% of the variation in the average grades is explained by the variation in everything else. The variables that have statistically significant impact on average grade are studytime (duh...), schoolsup, paid, and higher. 

In [ ]:
#2) Regression tree: 
library(rpart)
library(DMwR)# I will be relying heavily on the DMwR library that comes with Torgo, L. (2011) Data Mining with R. 
rt2<-rpart(avggrades~., data=d3norepeats[,1:30])
prettyTree(rt2)

According to the regression tree analysis, the variable that seems to be important in "higher" that indicates whether the student  wants to pursue higher education. The overwhelming majority of surveyed students would like to pursue higher education and their average grade (11.4/20) is significantly higher than the average grade of those who don't (8.47/20). Regression tree analysis reveals that mother's education is another important feature (interestingly, this feature did not come up as important in the linear regression model). Students whose mothers had at least secondary education had significantly higher grade (12.2) than the students whose mothers do not (their average grade was 11). I would like to take the first stab at  evaluating the relative predictive performance of the two models. Without getting into cross-validation, I'm at first interested in the normalized mean squared error of the two models. The following code does it. 

In [ ]:
#predictions
lm.predictions<-predict(lm2,d3norepeats)
rt.predictions<-predict(rt2,d3norepeats)
nmse.lm<-mean((lm.predictions-d3norepeats[,"avggrades"])^2)/mean((mean(d3norepeats$avggrades)-d3norepeats[,"avggrades"])^2)
nmse.rt<-mean((rt.predictions-d3norepeats[,"avggrades"])^2)/mean((mean(d3norepeats$avggrades)-d3norepeats[,"avggrades"])^2)
print(nmse.lm) #0.79
print(nmse.rt) #0.85

It seems that the linear model performs better than the regression tree.  The following model code shows the error scatter plots.

In [ ]:
lmpltdata1=data.frame(cbind(lm.predictions,d3norepeats[,"avggrades"]))
colnames(lmpltdata1)<-c("lm.predictions","avggrades")
rtpltdata1=data.frame(cbind(rt.predictions,d3norepeats[,"avggrades"]))
colnames(rtpltdata1)<-c("rt.predictions","avggrades")

d3norepeats$Dalc<-as.factor(d3norepeats$Dalc)

errplt.lt1=ggplot(lmpltdata1,aes(lm.predictions,avggrades))+
                  geom_point(aes(color=d3norepeats[,"Dalc"]))+
                  xlab("Predicted Grades (Linear Model)")+
                  ylab("Actual Grades")+
                  geom_abline(intercept=0,slope=1,color="#0066CC",size=1)+
                  #geom_smooth(method = "lm", se = FALSE)+
                  scale_colour_brewer(palette = "Set1",name = "Daily Alcohol \nConsumption")

errplt.rt1=ggplot(rtpltdata1,aes(rt.predictions,avggrades))+
  geom_point(aes(color=d3norepeats[,"Dalc"]))+
  xlab("Predicted Grades (Regression Tree)")+
  ylab("Actual Grades")+
  geom_abline(intercept=0,slope=1,color="#0066CC",size=1)+
  #geom_smooth(method = "lm", se = FALSE)+
  scale_colour_brewer(palette = "Set1",name = "Daily Alcohol \nConsumption")

grid.arrange(errplt.lt1,errplt.rt1,nrow=2)

In the above graphs, horizontal axes represent predicted grades while the vertical axes represent true grades. If the model is accurate in predicting actual grades then predicted grades must be equal to actual grades and thus the scatter points should line up along the 45 degree (blue) line. Unfortunately, as the NMSEs and error plots indicate, neither of the two models seems to do a decent job in predicting student average grades. Unsatisfied with how linear regression and regression tree models perform, I am going to take it up a notch and give a random forest a try.

In [ ]:
library(randomForest)
set.seed(4543)
rf2<-randomForest(avggrades~., data=d3norepeats[,1:30], ntree=500, importance=T)
rf.predictions<-predict(rf2,d3norepeats)
nmse.rf<-mean((rf.predictions-d3norepeats[,"avggrades"])^2)/mean((mean(d3norepeats$avggrades)-d3norepeats[,"avggrades"])^2)
print(nmse.rf) #0.2

NMSE of the random forest implementation is 0.2 and it is much, much lower than that of the linear and regression tree models. As a validation, I am going to obtain the error plot of the random forest and compare it with the error plots of the linear and regression tree models obtained above.

In [ ]:
#first combine the rf predictions and actual scores in a single data frame
rfpltdata1=data.frame(cbind(rf.predictions,d3norepeats[,"avggrades"]))
colnames(rfpltdata1)<-c("rf.predictions","avggrades")

# then create the error plot.
errplt.rf1<-ggplot(rfpltdata1,aes(rf.predictions,avggrades))+
  geom_point(aes(color=d3norepeats[,"Dalc"]))+
  xlab("Predicted Grades (Random Forest with 500 Trees)")+
  ylab("Actual Grades")+
  geom_abline(intercept=0,slope=1,color="#0066CC",size=1)+
  #geom_smooth(method = "lm", se = FALSE)+
  scale_colour_brewer(palette = "Set1",name = "Daily Alcohol \nConsumption")
#finally, plot the error plot from the random forest with the error plots of the linear and regression tree models.
grid.arrange(errplt.rf1, errplt.lt1,errplt.rt1,nrow=3)

Even though, random forest seems to systematically underpredict the grades of low grade earners and overpredict the grades of high grade earners, overall, random forest seems to be a much better predictor of average grades than either the linear regression or regression tree model. 10 x 5 fold cross validation that I run on my local machine confirms that out of the three models that I executed here (LM, RT, & RF), the RF model with 500 trees is indeed the best predictor of average student grades (somehow Kaggle's notebook environment keeps on reconnecting and is unable to run the cross-validation code in the notebook environment).  I plot the relative importance of all the features in the dataset as measured by the Random Forest with 500 trees.

In [ ]:
varImpPlot(rf2,type=1) # this metric is obtained by measuring the %increase in MSE of the model if the variable is removed

The top 10 most important variables that impact student average grades are:

 -  **higher**- wants to take higher education (binary: yes or no)
 - **Medu** - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade,        3 - secondary education or 4 – higher education)
 - **studytime** - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
 - **schoolsup**  - extra educational support (binary: yes or no)
 - **Dalc** - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
 - **goout**  - going out with friends (numeric: from 1 - very low to 5 - very high)
 - **Walc** - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
 - **reason** - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
 - **Fedu** - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
 - **sex** - student's sex (binary: 'F' - female or 'M' - male)

These results imply that both weekday and weekend alcohol consumption are important predictors of student average grades. Removing either of these two variables will increase the MSE of predictions by between 10-20%. 

What's interesting about these results is that some features that would be conventionally thought as important did not end up in the top ten list (I would speculate that the variables such as Pstatus, famsupport, famrel, & absences are among those) while some other variables (such as higher and Medu) turned out to be very important. 

In what follows, I am going to produce a partial dependence plot for each feature in the dataset (ranked by importance). Partial dependence plots give a graphical depiction of the marginal effect of a feature on the response. In this case, partial dependencies are produced by the best performing Random Forest model with 500 trees.  As the plots indicate, shifting from "very low" levels to just "low" levels of alcohol consumption on weekdays, will reduce the expected average grade from 11.22 to 10.88. Similarly, moving from "very low" levels to just "low" levels of alcohol consumption on weekends will reduce the average predicted grade from 11.19 to 11.17. 

As a conclusion, the impact of the most important 2 variables, "higher" and "Medu," on average student grade is as follows: 

 - **higher**: willingness to pursue higher education increases average predicted grade from 9.42 to 11.25. Thus, motivate your kids to pursue higher education is the best thing you can do to improve their grades in school!
 - **Medu**: increase in mother's education from none to more than secondary education increases predicted average grade from 10.8 to 11.5. Thus, advice to future parents (especially fathers): If you want your future children do well in school marry someone educated.

In [ ]:
imp <- importance(rf2)
impvar <- rownames(imp)[order(imp[, 1], decreasing=TRUE)]
op <- par(mfrow=c(2, 3))
for (i in seq_along(impvar)) {
  partialPlot(rf2, d3norepeats[,1:30], impvar[i], ,rug=TRUE, xlab=impvar[i],
              main=paste("Partial Dependence on", impvar[i]))
  abline(h=mean(d3norepeats$avggrades),col="red")
}
par(op)